In [1]:
directions = ['NW', 'N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
order_0 = ['N', 'S', 'W', 'E']

In [2]:
def load_matrix():
    with open('input23.txt') as csvfile:
        return [[c for c in line.strip()] for line in csvfile.readlines()]

In [3]:
def smallest_rectangle_containing_elves(matrix):
    min_x, max_x = min(line.index('#') for line in matrix if '#' in line), max(len(line) - line[::-1].index('#') - 1 for line in matrix if '#' in line)
    for i in range(len(matrix)):
        if '#' in matrix[i]: break
    min_y = i
    for i in range(len(matrix)-1, -1, -1):
        if '#' in matrix[i]: break
    max_y = i
    return min_y, max_y, min_x, max_x

In [4]:
def crop_retangle(matrix, coordinates):
    min_y, max_y, min_x, max_x = coordinates
    return [line[min_x:max_x+1] for line in matrix[min_y:max_y+1]]

In [5]:
def add_borders(matrix):
    return [['.']*(len(matrix[0])+2)] + [['.'] + line + ['.'] for line in matrix] + [['.']*(len(matrix[0])+2)]

In [6]:
def find_elves(matrix):
    elves = []
    for i in range(len(matrix)): elves.extend([(i, j) for j in range(len(matrix[i])) if matrix[i][j] == '#'])
    return elves

In [7]:
def adjacent_position(matrix, position, direction):
    next_position_y, next_position_x = position[:]
    if 'N' in direction:
        next_position_y -= 1
    if 'S' in direction:
        next_position_y += 1
    if 'W' in direction:
        next_position_x -= 1
    if 'E' in direction:
        next_position_x += 1
    return (next_position_y, next_position_x)

In [8]:
def access_matrix_position(matrix, position):
    return matrix[position[0]][position[1]]

In [9]:
def first_half_round(matrix, round_i = 0):
    order_i = [order_0[i % len(order_0)] for i in range(round_i, round_i+4)]
    first_half_propositions = {}
    
    for elf_position in find_elves(matrix):
        if '#' in [access_matrix_position(matrix, adjacent_position(matrix, elf_position, direction)) for direction in directions]:
            for orthogonal_direction in order_i:
                if '#' not in [access_matrix_position(matrix, adjacent_position(matrix, elf_position, direction)) for direction in directions if orthogonal_direction in direction]:
                    possible_adjacent_position = adjacent_position(matrix, elf_position, orthogonal_direction)
                    if possible_adjacent_position in first_half_propositions:
                        first_half_propositions[possible_adjacent_position].append(elf_position)
                        break
                    else:
                        first_half_propositions[possible_adjacent_position] = [elf_position]
                        break

    return first_half_propositions

In [10]:
def second_half_round(matrix, first_half_propositions):
    for possible_adjacent_position in first_half_propositions:
        if len(first_half_propositions[possible_adjacent_position]) == 1:
            elf_position = first_half_propositions[possible_adjacent_position].pop()
            matrix[elf_position[0]][elf_position[1]] = '.'
            matrix[possible_adjacent_position[0]][possible_adjacent_position[1]] = '#'
    return matrix

In [11]:
def no_elves_moves(first_half_propositions):
    for possible_adjacent_position in first_half_propositions:
        if len(first_half_propositions[possible_adjacent_position]) == 1:
            return False
    return True

In [12]:
def process_elves_positions(n_rounds=10, return_stochastic_round = False):
    matrix = load_matrix()
    i = 0
    while (return_stochastic_round or i < n_rounds):
        matrix = add_borders(crop_retangle(matrix, smallest_rectangle_containing_elves(matrix)))
        first_half_propositions = first_half_round(matrix, i)
        if return_stochastic_round and no_elves_moves(first_half_propositions): return i+1
        matrix = second_half_round(matrix, first_half_propositions)
        i+=1

    return crop_retangle(matrix, smallest_rectangle_containing_elves(matrix))

In [13]:
def empty_ground_tiles(matrix):
    return sum(line.count('.') for line in matrix)

Part 01

In [14]:
matrix = process_elves_positions(n_rounds = 10)
answer = empty_ground_tiles(matrix)
print(answer)

4208


Part 02

In [15]:
answer = process_elves_positions(return_stochastic_round = True)
print(answer)

1016
